In [4]:
import os
import matplotlib as mpl
import matplotlib.pyplot as plt
import scienceplots
import matplotlib.dates as mdates
import datetime as dt
import polars as pl
import matplotlib.pyplot as plt

from pathlib import Path
import sys

# Detect repo root: parent of ./notebooks
repo = Path.cwd().resolve()
if repo.name == "notebooks":
    repo = repo.parent
sys.path.append(str(repo))


from src.ploting.styles import set_plot_style
set_plot_style("prism_rain")
from src.parsing.runners import process_all_raw_recursive
from src.warehouse.db_build import build_warehouse, build_timeline_dataset
from src.parsing.runners import process_all_raw_recursive  # nombre según tu módulo

from src.warehouse.builder import build_parquet_from_raw_parallel


In [8]:
os.environ["POLARS_MAX_THREADS"] = "10"  


out = build_parquet_from_raw_parallel(
    raw_root       = repo / "data" / "00_raw",
    out_root       = repo / "data" / "02_warehouse",
    procedures_yaml= repo / "config" / "procedures.yml",  # uses your YAML
    overwrite      = True,
    verbose        = True,
    max_workers    = 8,   # tune for your disk/CPU
)
print(out)

[raw->parquet] scan /home/mphstphls/python-lab/data/00_raw … csv=30600
  … 200/30600 processed
  … 400/30600 processed
  … 600/30600 processed
  … 800/30600 processed
  … 1000/30600 processed
  … 1200/30600 processed
  … 1400/30600 processed
  … 1600/30600 processed
  … 1800/30600 processed
  … 2000/30600 processed
  … 2200/30600 processed
  … 2400/30600 processed
  … 2600/30600 processed
  … 2800/30600 processed
  … 3000/30600 processed
  … 3200/30600 processed
  … 3400/30600 processed
  … 3600/30600 processed
  … 3800/30600 processed
  … 4000/30600 processed
  … 4200/30600 processed
  … 4400/30600 processed
  … 4600/30600 processed
  … 4800/30600 processed
  … 5000/30600 processed
  … 5200/30600 processed
  … 5400/30600 processed
  … 5600/30600 processed
  … 5800/30600 processed
  … 6000/30600 processed
  … 6200/30600 processed
  … 6400/30600 processed
  … 6600/30600 processed
  … 6800/30600 processed
  … 7000/30600 processed
  … 7200/30600 processed
  … 7400/30600 processed
  … 7600

In [10]:
import matplotlib.pyplot as plt
import polars as pl

lf = pl.scan_parquet("data/02_warehouse/raw_measurements/**/part-*.parquet")

counts = (
    lf.group_by_dynamic("start_dt", every="1d")   # group by day
      .agg(pl.len().alias("n"))
      .sort("start_dt")
      .collect()
)

# Convert columns to lists
days = counts["start_dt"].to_list()
vals = counts["n"].to_list()

fig, ax = plt.subplots(figsize=(12, 6))
ax.bar(days, vals, width=0.8)
ax.set_title("Experiments per day")
ax.set_xlabel("Date")
ax.set_ylabel("Count")
fig.autofmt_xdate()
plt.show()


ComputeError: expected at least 1 source

This error occurred with the following context stack:
	[1] 'parquet scan'
	[2] 'group_by'
	[3] 'select'
	[4] 'sink'
